# Indización y selección de datos

En el [Capítulo 2](https://colab.research.google.com/drive/1cvu7DvxbQhK7GLgH6fwRFC--v7hSLfkh?usp=sharing), vimos en detalle los métodos y herramientas para acceder, establecer y modificar valores en los arrays de NumPy.
Estos incluyen la indexación (por ejemplo, ``arr[2, 1]``), el corte (por ejemplo, ``arr[:, 1:5]``), el enmascaramiento (por ejemplo, ``arr[arr > 0]``), la indexación de lujo (por ejemplo, ``arr[0, [1, 5]]``), y combinaciones de los mismos (por ejemplo, ``arr[:, [1, 5]]``).
Aquí veremos medios similares para acceder y modificar valores en los objetos Pandas ``Series`` y ``DataFrame``.
Si has utilizado los patrones de NumPy, los patrones correspondientes en Pandas te resultarán muy familiares, aunque hay algunas peculiaridades que debes tener en cuenta.

Empezaremos con el caso simple del objeto ``Series`` unidimensional, y luego pasaremos al más complicado objeto ``DataFrame`` bidimensional.

## Selección de Datos en Series

Como vimos en la sección anterior, un objeto ``Series`` actúa de muchas maneras como un array unidimensional de NumPy, y de muchas maneras como un diccionario estándar de Python.
Si tenemos en cuenta estas dos analogías superpuestas, nos ayudará a entender los patrones de indexación y selección de datos en estos arrays.

### Series como diccionario

Al igual que un diccionario, el objeto ``Series`` proporciona un mapeo de una colección de claves a una colección de valores:

In [ ]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [ ]:
data['b']

0.5

También podemos utilizar expresiones y métodos de Python tipo diccionario para examinar las claves/índices y los valores:

In [ ]:
'a' in data

True

In [ ]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [ ]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Los objetos ``Series`` pueden incluso modificarse con una sintaxis similar a la de un diccionario.
Al igual que se puede ampliar un diccionario asignando una nueva clave, se puede ampliar una ``Serie`` asignando un nuevo valor de índice:

In [ ]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas está tomando decisiones sobre la disposición de la memoria y la copia de datos que podría tener lugar; el usuario generalmente no necesita preocuparse por estas cuestiones.

### Series como array unidimensional

Una ``Series`` se basa en esta interfaz tipo diccionario y proporciona una selección de elementos al estilo de las matrices mediante los mismos mecanismos básicos que las matrices de NumPy, es decir, *slices*, *masking* y *fancy indexing*.
Los ejemplos son los siguientes:

In [ ]:
# slicing por índice explícito
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [ ]:
# slicing por integer indexing implícito
data[0:2]

a    0.25
b    0.50
dtype: float64

In [ ]:
# masking
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [ ]:
# fancy indexing
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

Entre ellos, el corte puede ser la fuente de mayor confusión.
Observe que cuando se corta con un índice explícito (es decir, ``datos['a':'c']``), el índice final está *incluido* en el corte, mientras que cuando se corta con un índice implícito (es decir, ``datos[0:2]``), el índice final está *excluido* del corte.

### Indexadores: loc, iloc y ix

Estas convenciones de corte e indexación pueden ser una fuente de confusión.
Por ejemplo, si tu ``Serie`` tiene un índice entero explícito, una operación de indexación como ``datos[1]`` utilizará los índices explícitos, mientras que una operación de corte como ``datos[1:3]`` utilizará el índice implícito de estilo Python.

In [ ]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [ ]:
# índice explicito con indexing
data[1]

'a'

In [ ]:
# índice implicito co slicing
data[1:3]

3    b
5    c
dtype: object

Debido a esta potencial confusión en el caso de los índices enteros, Pandas proporciona algunos atributos *indexadores* especiales que exponen explícitamente ciertos esquemas de indexación.
No se trata de métodos funcionales, sino de atributos que exponen una interfaz de corte particular para los datos de la ``Series``.

En primer lugar, el atributo ``loc`` permite la indexación y el corte que siempre hace referencia al índice explícito:

In [ ]:
data.loc[1]

'a'

In [ ]:
data.loc[1:3]

1    a
3    b
dtype: object

El atributo ``iloc`` permite la indexación y el corte que siempre hace referencia al índice implícito de estilo Python:

In [ ]:
data.iloc[1]

'b'

In [ ]:
data.iloc[1:3]

3    b
5    c
dtype: object

Un tercer atributo de indexación, ``ix``, es un híbrido de los dos, y para los objetos ``Series`` es equivalente a la indexación estándar basada en ``[]``.
El propósito del indexador ``ix`` se hará más evidente en el contexto de los objetos ``DataFrame``, que discutiremos en un momento.

Un principio rector del código Python es que "lo explícito es mejor que lo implícito".
La naturaleza explícita de ``loc`` y ``iloc`` los hace muy útiles para mantener un código limpio y legible; especialmente en el caso de los índices de enteros, recomiendo usarlos tanto para hacer el código más fácil de leer y entender, como para prevenir errores sutiles debido a la convención mixta de indexación/corte.

## Selección de datos en DataFrame

Recordemos que un ``DataFrame`` actúa en muchos aspectos como un array bidimensional o estructurado, y en otros como un diccionario de estructuras ``Series`` que comparten el mismo índice.
Estas analogías pueden ser útiles para tener en cuenta mientras exploramos la selección de datos dentro de esta estructura.

### DataFrame como diccionario

La primera analogía que consideraremos es el ``DataFrame`` como diccionario de objetos ``Series`` relacionados.
Volvamos a nuestro ejemplo de áreas y poblaciones de los estados:

In [ ]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
data = pd.DataFrame({'area':area, 'pop':pop})
data

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135
New York,141297,19651127
Texas,695662,26448193


Se puede acceder a las ``Series`` individuales que componen las columnas del ``DataFrame`` a través de la indexación de estilo diccionario del nombre de la columna:

In [ ]:
data['area']

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

De forma equivalente, podemos utilizar un acceso de tipo atributo con nombres de columna que sean cadenas:

In [ ]:
data.area

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

Este acceso a la columna de estilo atributo accede en realidad al mismo objeto que el acceso de estilo diccionario:

In [ ]:
data.area is data['area']

True

Aunque esta es una abreviatura útil, tenga en cuenta que no funciona en todos los casos.
Por ejemplo, si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con los métodos de ``DataFrame``, este acceso tipo atributo no es posible.
Por ejemplo, el ``DataFrame`` tiene un método ``pop()``, así que ``data.pop`` apuntará a éste en lugar de a la columna ``"pop"``:

In [ ]:
data.pop is data['pop']

False

En particular, debe evitar la tentación de intentar la asignación de columnas a través de atributos (es decir, utilice ``datos['pop'] = z`` en lugar de ``datos.pop = z``).

Al igual que con los objetos ``Series`` discutidos anteriormente, esta sintaxis de estilo diccionario también se puede utilizar para modificar el objeto, en este caso añadiendo una nueva columna:

In [ ]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740


Esto muestra una vista previa de la sintaxis directa de la aritmética elemento a elemento entre los objetos ``Series``; profundizaremos en esto en [Operando con datos en Pandas](03.03-Operaciones-en-Pandas.ipynb).

### DataFrame como array bidimensional

Como se ha mencionado anteriormente, también podemos ver el ``DataFrame`` como un array bidimensional mejorado.
Podemos examinar la matriz de datos subyacente utilizando el atributo ``values``:

In [ ]:
data.values

array([[  4.23967000e+05,   3.83325210e+07,   9.04139261e+01],
       [  1.70312000e+05,   1.95528600e+07,   1.14806121e+02],
       [  1.49995000e+05,   1.28821350e+07,   8.58837628e+01],
       [  1.41297000e+05,   1.96511270e+07,   1.39076746e+02],
       [  6.95662000e+05,   2.64481930e+07,   3.80187404e+01]])

Con esta imagen en mente, se pueden hacer muchas observaciones familiares de tipo array en el propio ``DataFrame``.
Por ejemplo, podemos transponer el ``DataFrame`` completo para intercambiar filas y columnas:

In [ ]:
data.T

,California,Florida,Illinois,New York,Texas
area,4.239670e+05,1.703120e+05,1.499950e+05,1.412970e+05,6.956620e+05
pop,3.833252e+07,1.955286e+07,1.288214e+07,1.965113e+07,2.644819e+07
density,9.041393e+01,1.148061e+02,8.588376e+01,1.390767e+02,3.801874e+01


Sin embargo, cuando se trata de la indexación de los objetos ``DataFrame``, está claro que la indexación de las columnas al estilo de un diccionario impide nuestra capacidad de tratarlo simplemente como un array de NumPy.
En particular, al pasar un solo índice a un array se accede a una fila:

In [ ]:
data.values[0]

array([  4.23967000e+05,   3.83325210e+07,   9.04139261e+01])

y pasando un único "index" a un ``DataFrame`` se accede a una columna:

In [ ]:
data['area']

California    423967
Florida       170312
Illinois      149995
New York      141297
Texas         695662
Name: area, dtype: int64

Por lo tanto, para la indexación estilo array, necesitamos otra convención.
Aquí Pandas vuelve a utilizar los indexadores ``loc``, ``iloc`` y ``ix`` mencionados anteriormente.
Usando el indexador ``iloc``, podemos indexar el array subyacente como si fuera un simple array de NumPy (usando el índice implícito de estilo Python), pero el índice ``DataFrame`` y las etiquetas de las columnas se mantienen en el resultado:

In [ ]:
data.iloc[:3, :2]

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


Del mismo modo, utilizando el indexador ``loc`` podemos indexar los datos subyacentes en un estilo similar al de los arrays pero utilizando el índice explícito y los nombres de las columnas:

In [ ]:
data.loc[:'Illinois', :'pop']

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


El indexer ``ix'' permite un híbrido de estos dos enfoques:

In [ ]:
data.ix[:3, :'pop']

,area,pop
California,423967,38332521
Florida,170312,19552860
Illinois,149995,12882135


Tenga en cuenta que para los índices enteros, el indexer ``ix`` está sujeto a las mismas fuentes potenciales de confusión que se discutieron para los objetos ``Series`` indexados por enteros.

Cualquiera de los patrones de acceso a los datos de estilo NumPy puede ser utilizado dentro de estos indexadores.
Por ejemplo, en el indexador ``loc`` podemos combinar el enmascaramiento y la indexación de fantasía como en lo siguiente:

In [ ]:
data.loc[data.density > 100, ['pop', 'density']]

,pop,density
Florida,19552860,114.806121
New York,19651127,139.076746


Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; esto se hace de la manera estándar a la que podría estar acostumbrado de trabajar con NumPy:

In [ ]:
data.iloc[0, 2] = 90
data

,area,pop,density
California,423967,38332521,90.000000
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763
New York,141297,19651127,139.076746
Texas,695662,26448193,38.018740


Para mejorar tu fluidez en la manipulación de datos de Pandas, te sugiero que pases algún tiempo con un simple ``DataFrame`` y explores los tipos de indexación, corte, enmascaramiento e indexación de fantasía que permiten estos diversos enfoques de indexación.

### Convenciones de indexación adicionales

Hay un par de convenciones de indexación adicionales que pueden parecer contrarias a la discusión anterior, pero que sin embargo pueden ser muy útiles en la práctica.
En primer lugar, mientras que *indexar* se refiere a las columnas, *cortar* se refiere a las filas:

In [ ]:
data['Florida':'Illinois']

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Estos cortes también pueden referirse a las filas por número en lugar de por índice:

In [ ]:
data[1:3]

,area,pop,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Del mismo modo, las operaciones de enmascaramiento directo también se interpretan por filas en lugar de por columnas:

In [ ]:
data[data.density > 100]

,area,pop,density
Florida,170312,19552860,114.806121
New York,141297,19651127,139.076746


Estas dos convenciones son sintácticamente similares a las de un array de NumPy, y aunque no se ajusten precisamente al molde de las convenciones de Pandas, son sin embargo bastante útiles en la práctica.